In [1]:
import numpy as np
import re
#import fasttext
import time
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import subprocess
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
f_news = []

with open('datasets/news_fasttext_train.txt') as f:
    for line in f:
        f_news.append(f.readline())

print(f_news[-2].split('\t')[0].replace('\xa0', '').replace('  ', '').split('\u3000 \u3000'))

['数码 通及 合 和 实业 回购 股份 ', ' 新浪 财经 讯 12 月 01 日 消息 ， 数码 通 ( 00315 - HK ) 申报 回购 33 万股 ， 每股 作价 5.12 元至 5.2 元 ， 涉资 170.41 万元 。 ', ' 合 和 实业 ( 00054 - HK ) 申报 回购 1 , 500 股 ， 每股 作价 24.3 元 ， 涉 3.65 万元 。已有 _ COUNT _ 条 评论 我要 评论']


In [14]:
with open('datasets/news_fasttext_train.txt') as origin:
    with open('datasets/news_raw_nolabel.txt', 'w') as f:
        for line in origin:
            temp = line.split('\t')[0].replace('\xa0', '').replace('  ', '').split('\u3000 \u3000')
            for i in range(len(temp)):
                temp[i] = temp[i].replace('\u3000', '')
                f.write("{}\n".format(temp[i]))

In [17]:
tokens = 0
i = 0
with open('datasets/news_raw_nolabel.txt', 'r') as f:
    for line in f:
        tokens += len(line.split())

print("Total number of tokens : " + str(tokens))

Total number of tokens : 62014273


In [15]:
with open('pre_trained_vectors/raw_300/ft_raw_skipgram_300.vec', 'r') as f:
    with open('pre_trained_vectors/raw_300/ft_5_vocab.txt', 'w+') as file:
        file.write('<S>\n</S>\n')
        _ = f.readline()
        for line in f:
            temp = line.split()
            file.write('{}\n'.format(temp[0]))            

In [17]:
with open('pre_trained_vectors/raw_300/raw_5_vocab.txt', 'r') as f:
    with open('pre_trained_vectors/raw_300/glove_5_vocab.txt', 'w+') as file:
        file.write('<S>\n</S>\n<UNK>\n')
        for line in f:
            temp = line.split()
            file.write('{}\n'.format(temp[0]))  

Transfer the txt vectors to hdf5

In [18]:
import h5py
import numpy as np

n_token = 0
with open('/home/chenxin/WordEmbedding/pre_trained_vectors/raw_300/ft_5_vocab.txt', 'r') as f:
    for line in f:
        n_token += 1
        
embeddings = np.zeros((n_token, 300))
i = 2
with open('/home/chenxin/WordEmbedding/pre_trained_vectors/raw_300/ft_raw_skipgram_300.vec', 'r') as f:
    _ = f.readline()
    for line in f:
        temp = line.split()
        vec = np.array(temp[1:])
        embeddings[i] = vec
        i += 1


ValueError: could not broadcast input array from shape (299) into shape (300)

In [19]:
with open('datasets/news_raw_nolabel.txt', 'r') as f:
    for line in f:
        if '\u3000' in line:
            print(str(line))

In [23]:
i = 2
with open('/home/chenxin/WordEmbedding/pre_trained_vectors/raw_300/ft_5_vocab.txt', 'r') as f:
    for line in f:
        

ValueError: could not broadcast input array from shape (299) into shape (300)

In [27]:
print(line.encode('utf-8'))

b'\xe2\x80\x83 0.86293 0.22522 -0.79965 0.4992 0.1027 0.22174 -0.40173 -0.31652 -0.048179 0.052495 0.73216 1.0513 -0.030755 0.32309 -0.53907 0.68331 0.098927 -0.60278 -0.12234 -0.22029 -0.0085215 0.63536 0.35803 0.57202 -0.63249 0.1675 0.24349 0.26912 0.2611 0.23714 0.0058989 0.067044 -0.50479 -0.33798 -0.63998 0.50098 -0.061128 -0.49068 -0.2643 -0.10708 -0.44271 -0.13776 -0.49906 0.36042 -0.057518 0.17255 0.33322 -0.65009 -0.68437 0.27279 -0.5577 -0.13477 0.26014 -0.29837 0.58462 -0.25372 0.068013 -0.5997 0.15197 -0.39636 0.20379 -0.01857 0.047061 -0.038932 0.11172 0.0085653 0.099622 -0.043242 -0.040492 -0.69145 0.20698 0.19222 0.0027948 -0.11879 -0.38068 -0.62065 0.86715 0.60022 -0.37306 -0.01661 -0.1803 0.31454 0.16876 -0.625 0.25383 0.078141 -0.14337 -0.36476 -0.19892 0.061985 -0.28267 0.3232 0.55201 0.0040214 0.42924 -0.15917 -0.71128 -0.063316 0.42834 0.070467 0.28878 -0.14301 -0.047324 0.0095609 -0.60631 0.72252 0.7909 -0.020283 0.8824 -0.15433 0.35441 0.31628 -0.4513 0.037693 -